# Reddit news

In [2]:
import pandas as pd
import sqlite3

In [22]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
kolonas = ['date']+['top'+str(i) for i in range(1,31)]
print(kolonas)

In [ ]:
df = pd.DataFrame(columns = kolonas)

In [ ]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()

In [ ]:
sql = ("Select date(moment) as date, title from top30 where date = '2021-05-05'")
z = pd.read_sql_query(sql,connection)

In [ ]:
for d in z.date.unique():
    atbilde =[]
    atbilde.append(d)
    #print(atbilde)
    for title in z.title[z.date == d]:
        atbilde.append(title)
        
    df.loc[len(df)] = atbilde

In [ ]:
df.to_sql('main', connection, if_exists='append',index=False)

In [ ]:
atbilde = []
atbilde.append(z.date[0])
#print(atbilde)

for title in z.title:
    atbilde.append(title)
        
print(atbilde)
df.loc[len(df)] = atbilde

# delete

connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("DELETE from top30 where moment  between '2021-03-30 11:17:11.915369' and '2021-03-30 11:18:48.647641'")
cursor.execute(sql)
connection.commit()

# Yahoo finance

In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
df = pd.DataFrame(columns = ['date','SP500_Open', 'SP500_High', 'SP500_Low', 'SP500_Close', 'SP500_Adj_Close', 'SP500_Volume'])
df

In [ ]:
str([str('SP500_')+i for i in df.columns])

## S&P 500 (^GSPC)
SNP - SNP Real Time Price. Currency in USD

In [3]:
url = "https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC"

In [4]:
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

In [5]:
answer = []
for i in soup.find_all('td')[0:7]:
    #print(i.span)
    print(i.span.text)
    answer.append(i.span.text)
    #print(len(answer)    

May 07, 2021
4,210.34
4,238.04
4,201.64
4,232.60
4,232.60
4,013,060,000


for i in range(0, int(len(answer))//7):
    atbilde = []
    #print(i)
    for j in range (0,7):
        if j == 0:
            atbilde.append(pd.to_datetime(answer[i*7+j], infer_datetime_format=True))
        else:
            atbilde.append(answer[i*7+j])
    df.loc[len(df)] = atbilde

In [ ]:
connection = sqlite3.connect('MSC/MSC.db')
cursor = connection.cursor()
sql = ("Select date(moment) as date, title from top30 where moment>date('now')")
z = pd.read_sql_query(sql,connection)

atbilde = []
columnnames = "'date', 'top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8', 'top9', 'top10', 'top11', 'top12', 'top13', 'top14', 'top15', 'top16', 'top17', 'top18', 'top19', 'top20', 'top21', 'top22', 'top23', 'top24', 'top25', 'top26', 'top27', 'top28', 'top29', top30"
insertintotable = '?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?'
atbilde = [z.date[0]] +[i for i in  z.title]

connection = sqlite3.connect('MSC/MSC.db')
cursor = connection.cursor()
sql2 = f'cursor.execute("insert INTO  main ({columnnames}) VALUES ({insertintotable})", {atbilde})'
eval(sql2)
connection.commit()
connection.close()
print("Appended to MAIN")

In [95]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("Select * from DJI")
z = pd.read_sql_query(sql,connection).columns
z

Index(['date', 'DJI_Open', 'DJI_High', 'DJI_Low', 'DJI_Close', 'DJI_Adj_Close',
       'DJI_Volume'],
      dtype='object')

In [93]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
sql = ("Select max(date) as date from SP500")
z = pd.read_sql_query(sql,connection)
latest_date = z.date[0]

##SP500
url = "https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC"
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

answer = []
if str(pd.to_datetime(soup.find_all('td')[0].text, infer_datetime_format=True)) == latest_date:
    print('Stop. This data point is already in the data base')
else:
    print('Lets get scraping')
    answer.append(str(pd.to_datetime(soup.find_all('td')[0].text, infer_datetime_format=True)))
    answer = answer + [i.span.text for i in soup.find_all('td')[1:7]]
    
    columnnames = "'Date', 'SP500_Open', 'SP500_High', 'SP500_Low', 'SP500_Close','SP500_Adj_Close', 'SP500_Volume'"
    insertintotable = '?,?,?,?,?,?,?'
    
    connection = sqlite3.connect('MSC/MSC.db')
    cursor = connection.cursor()
    sql2 = f'cursor.execute("insert INTO  SP500 ({columnnames}) VALUES ({insertintotable})", {answer})'
    eval(sql2)
    connection.commit()
    connection.close()
    print(f"{answer[0]} appended to SP500")
    
    
    ## DJI
    url = "https://finance.yahoo.com/quote/%5EDJI/history?p=%5EDJI"
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data)
    
    answer = []
    answer.append(str(pd.to_datetime(soup.find_all('td')[0].text, infer_datetime_format=True)))
    answer = answer + [i.span.text for i in soup.find_all('td')[1:7]]
    
    columnnames = "'date', 'SP500_Open', 'SP500_High', 'SP500_Low', 'SP500_Close','SP500_Adj_Close', 'SP500_Volume'"
    insertintotable = '?,?,?,?,?,?,?'
    
    connection = sqlite3.connect('MSC/MSC.db')
    cursor = connection.cursor()
    sql2 = f'cursor.execute("insert INTO  SP500 ({columnnames}) VALUES ({insertintotable})", {answer})'
    eval(sql2)
    connection.commit()
    connection.close()
    print(f"{answer[0]} appended to DJI")
    
    
answer

Stop. This data point is already in the data base


[]

In [ ]:
df.to_sql('SP500', connection, if_exists='append',index=False)

## Dow Jones Industrial Average (^DJI)
DJI - DJI Real Time Price. Currency in USD

In [ ]:
url = "https://finance.yahoo.com/quote/%5EDJI/history?p=%5EDJI"

In [ ]:
df = pd.DataFrame(columns = ['date', 'DJI_Open', 'DJI_High', 'DJI_Low', 'DJI_Close', 'DJI_Adj_Close', 'DJI_Volume'])
df

In [ ]:
columns_new = [str('DJI_')+i for i in df.columns]

In [ ]:
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

In [ ]:
answer = []
for i in soup.find_all('td'):
    #print(i.span)
    #print(i.span.text)
    answer.append(i.span.text)
    #print(len(answer)  

In [ ]:
for i in range(0, int(len(answer))//7):
    atbilde = []
    #print(i)
    for j in range (0,7):
        if j == 0:
            atbilde.append(pd.to_datetime(answer[i*7+j], infer_datetime_format=True))
        else:
            atbilde.append(answer[i*7+j])
    df.loc[len(df)] = atbilde

In [ ]:
df

In [ ]:
df.to_sql('DJI', connection, if_exists='append',index=False)

# Merge

In [214]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()

sql = ('''Select 
            main.date,
            top1,top2,top3,top4,top5,
            top6,top7,top8,top9,top10,
            top11,top12,top13,top14,top15,
            top16,top17,top18,top19,top20,
            top21,top22,top23,top24,top25,
            top26,top27,top28,top29,top30,
            
            DJI_Open,
            DJI_High,
            DJI_Low,
            DJI_Close,
            DJI_Adj_Close,
            
            SP500_Open,
            SP500_High,
            SP500_Low,
            SP500_Close,
            SP500_Adj_Close

            from main
            left join DJI on
            main.date = date(DJI.Date)
            left join SP500 on
            main.date = date(SP500.Date)

       ''')
df = pd.read_sql_query(sql,connection)
df.head()

,date,top1,top2,top3,top4,top5,top6,top7,top8,top9,...,DJI_Open,DJI_High,DJI_Low,DJI_Close,DJI_Adj_Close,SP500_Open,SP500_High,SP500_Low,SP500_Close,SP500_Adj_Close
0,2021-03-09,China breaching every act in genocide conventi...,"Around 1,000 women gather in Istanbul to prote...",Greta Thunberg says Biden isn’t doing ‘nearly ...,Humans have 'destroyed or degraded' two-thirds...,Biden and Europe allies worry Israel is prepar...,Syrian President Assad tests positive for COVI...,Covid-19: 'Certain inevitability' that visitor...,First Nation asks UN expert to probe systemic ...,Brazil Supreme Court judge annuls ex-president...,...,"31,892.35","32,150.32","31,822.64","31,832.74","31,832.74","3,851.93","3,903.76","3,851.93","3,875.44","3,875.44"
1,2021-03-10,Scientists Discover Chunk of Protoplanet Older...,'Shoot till they are dead': Some Myanmar polic...,"Biden to meet virtually with leaders of Japan,...","Up to 1,000 Christian, Hindu and Sikh girls ar...","Papuan island inhabitants, whose island was of...",China’s appetite for meat fades as vegan revol...,French nuclear tests infected 'almost entire P...,Sex between coaches and teenagers in their car...,'Right to repair' law to come into effect this...,...,"31,906.96","32,389.50","31,906.96","32,297.02","32,297.02","3,891.99","3,917.35","3,885.73","3,898.81","3,898.81"
2,2021-03-11,Conspiracy Theorist Who Stormed Trudeau’s Prop...,Biden says he’ll share surplus US vaccines wit...,Mexico Passes Bill to Legalize Cannabis,"India now as autocratic as Pakistan, worse tha...",Thousands in India demand top judge resign ove...,"Hong Kong: China approves ""patriotic"" plan to ...",Myanmar’s military on ‘killing spree’ against ...,The Pfizer-BioNTech coronavirus vaccine 97% ef...,Myanmar's searing smartphone images flood a wa...,...,"32,354.50","32,661.59","32,345.70","32,485.59","32,485.59","3,915.54","3,960.27","3,915.54","3,939.34","3,939.34"
3,2021-03-12,"Us secretary of state calls Taiwan ""country""","Myanmar's military is likely committing ""crime...",Vaccination of Toronto's homeless well underwa...,Novavax vaccine 96% effective against original...,"Kremlin critic Navalny moved from jail, wherea...",Global warming leads to fish die off in South ...,12 protesters killed in Myanmar as military de...,Drug clan leader testifies he bribed Honduran ...,Canadian health officials watching probes of A...,...,"32,462.40","32,793.32","32,462.40","32,778.64","32,778.64","3,924.52","3,944.99","3,915.21","3,943.34","3,943.34"
4,2021-03-13,Unions call strike at Amazon's logistic operat...,Sri Lanka will ban the wearing of the burqa an...,U.S. Offers Protected Status For People From M...,Japan detects new coronavirus variant from tra...,Odisha Man Creates 25 Acres of Mangrove Forest...,Sarah Everard: Met Police officer charged with...,Myanmar Protest Leader Wants The Military Out ...,"Russia detains around 150 people, including le...",'Covid is taking over': Brazil plunges into de...,...,None,None,None,None,None,None,None,None,None,None


In [215]:
df = df[['date', 'top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8',
       'top9', 'top10', 'top11', 'top12', 'top13', 'top14', 'top15', 'top16',
       'top17', 'top18', 'top19', 'top20', 'top21', 'top22', 'top23', 'top24',
       'top25', 'top26', 'top27', 'top28', 'top29', 'top30']]

df.isnull().sum()

In [97]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

In [216]:
Anakin = SentimentIntensityAnalyzer()
Anakin.polarity_scores(" ")

{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}

In [217]:
def detect_subjectivity(text):
    '''
    subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
    '''
    return TextBlob(text).sentiment.subjectivity

detect_subjectivity(" ") #should return 0



0.0

In [218]:
start_vect=time.time()
#get the name of the headline columns
cols = []
for i in range(1,31):
    col = ("top{}".format(i))
    cols.append(col)
    
for col in cols:
    df[col] = df[col].astype(str) # Make sure data is treated as a string
    df[col+'_comp']= df[col].apply(lambda x:Anakin.polarity_scores(x)['compound'])
    df[col+'_sub'] = df[col].apply(detect_subjectivity)
    print("{} Done".format(col))
    
print("VADER: Vaderization completed after %0.2f Minutes"%((time.time() - start_vect)/60))


top1 Done
top2 Done
top3 Done
top4 Done
top5 Done
top6 Done
top7 Done
top8 Done
top9 Done
top10 Done
top11 Done
top12 Done
top13 Done
top14 Done
top15 Done
top16 Done
top17 Done
top18 Done
top19 Done
top20 Done
top21 Done
top22 Done
top23 Done
top24 Done
top25 Done
top26 Done
top27 Done
top28 Done
top29 Done
top30 Done
VADER: Vaderization completed after 0.04 Minutes


In [219]:
#the text isn't required anymore
df = df.drop(cols,axis=1)
df.head()

,date,top1_comp,top1_sub,top2_comp,top2_sub,top3_comp,top3_sub,top4_comp,top4_sub,top5_comp,...,top26_comp,top26_sub,top27_comp,top27_sub,top28_comp,top28_sub,top29_comp,top29_sub,top30_comp,top30_sub
0,2021-03-09,0.1280,0.200000,-0.2500,0.0,0.0000,0.500000,-0.5719,0.750000,-0.6369,...,-0.5267,0.238095,0.0000,0.400000,0.3818,0.000,0.3400,0.256250,-0.3612,0.250000
1,2021-03-10,0.0000,0.333333,-0.6249,0.4,0.1779,0.211111,-0.7430,0.100000,-0.7636,...,-0.7003,0.466667,-0.3818,0.475000,-0.4215,0.300,0.0000,1.000000,-0.3400,0.000000
2,2021-03-11,-0.5423,0.000000,0.2960,0.5,0.0000,0.000000,-0.4767,0.600000,-0.9557,...,0.0000,0.000000,-0.4588,0.333333,-0.5267,0.325,-0.8860,0.000000,-0.5574,0.214286
3,2021-03-12,0.0000,0.000000,0.2732,0.5,0.2732,0.000000,0.6597,0.775000,-0.2732,...,0.0000,0.454545,-0.3182,0.000000,-0.2960,0.000,0.1779,0.411111,-0.8360,0.000000
4,2021-03-13,-0.1280,0.000000,-0.5574,0.5,0.4404,0.144444,0.0000,0.454545,0.2732,...,-0.7003,0.000000,0.0000,0.333333,-0.7506,0.400,0.5859,0.225000,0.0000,0.000000


In [173]:
import numpy as np

In [224]:
len(df)

63

In [229]:
comp_cols = [col + "_comp" for col in cols]

w = np.arange(1,31,1).tolist()
w.reverse()

weighted_comp = []
max_comp = []
min_comp = []
for i in range(0,len(df)):
    a = df.loc[i,comp_cols].tolist()
    weighted_comp.append(np.average(a, weights=w))
    max_comp.append(max(a))
    min_comp.append(min(a))

In [230]:
df['compound_mean'] = weighted_comp
df['compound_max'] = max_comp
df['compound_min'] = min_comp

sub_cols = [col + "_sub" for col in cols]

weighted_sub = []
max_sub = []
min_sub = []
for i in range(0,len(df)):
    a = df.loc[i,sub_cols].tolist()
    weighted_sub.append(np.average(a, weights=w))
    max_sub.append(max(a))
    min_sub.append(min(a))

df['subjectivity_mean'] = weighted_sub
df['subjectivity_max'] = max_sub
df['subjectivity_min'] = min_sub

to_drop = sub_cols+comp_cols
df = df.drop(to_drop, axis=1)

In [ ]:
import plotly.graph_objects as go

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=df.Date, y=df.subjectivity_mean,
                    mode='lines',
                    name='Mean'))
fig3.add_trace(go.Scatter(x=df.Date, y=df.subjectivity_min,
                    mode='lines',
                    name='Min'))
fig3.add_trace(go.Scatter(x=df.Date, y=df.subjectivity_max,
                    mode='lines',
                    name='Max'))
title = []
title.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Development of subjectivity score',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig3.update_layout(xaxis_title='Date',
                   yaxis_title='Subjectivity score',
                  annotations=title)
fig3.show()

In [234]:
pip install plotly

     |████████████████████████████████| 13.2 MB 9.4 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=bfcc9185c7e7d4b098634ce0259b160541ee53287cd40a35475bbb8ebf13f5d3
  Stored in directory: /Users/Edite/Library/Caches/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
You should consider upgrading via the '/Users/Edite/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
